# Analyzing results from ULTRA prediction

## Load data

In [37]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

datadir = "../data"
dataset = "hetionet"
outdir = "../outputs"
res_name = "ultra_50g-Hetionet.csv"

train_df = pd.read_csv(f"{datadir}/{dataset}/train.txt", sep="\t", names=["source", "metaedge", "target"])
val_df = pd.read_csv(f"{datadir}/{dataset}/valid.txt", sep="\t", names=["source", "metaedge", "target"])
test_df = pd.read_csv(f"{datadir}/{dataset}/test.txt", sep="\t", names=["source", "metaedge", "target"])

pred_df = pd.read_csv(f"{outdir}/{res_name}")

In [38]:
def print_data_stats(df):
    print("Edge number: {}".format(len(df)))
    nodes = list(set(df["source"].tolist() + df["target"].to_list()))
    df["source_type"] = df["source"].str.split("::", expand=True)[0]
    df["target_type"] = df["target"].str.split("::", expand=True)[0]
    print("Node number: {}".format(len(nodes)))
    node_types = list(set(df["source_type"].tolist() + df["target_type"].to_list()))
    print("Node type number: {}".format(len(node_types)))


In [39]:
print("TRAIN:")
print_data_stats(train_df.copy())
print("-"*20)
print("\nVALID:")
print_data_stats(val_df.copy())
print("-"*20)
print("\nTEST:")
print_data_stats(test_df.copy())

TRAIN:
Edge number: 2025177
Node number: 44907
Node type number: 11
--------------------

VALID:
Edge number: 112510
Node number: 29250
Node type number: 11
--------------------

TEST:
Edge number: 112510
Node number: 29203
Node type number: 11


## New links

In [43]:
new_links_q1 = pred_df[["source", "metaedge", "target_pred"]]
new_links_q1.rename({"target_pred": "target"}, axis=1, inplace=True)
new_links_q2 = pred_df[["source_pred", "metaedge", "target"]]
new_links_q2.rename({"source_pred": "source"}, axis=1, inplace=True)
new_links = pd.concat([new_links_q1, new_links_q2], ignore_index=True)
print("Before removing duplications: {}".format(len(new_links)))
new_links.drop_duplicates(inplace=True, keep="first")
new_links.reset_index(inplace=True, drop=True)
print("Number of links recontructed from ULTRA: {}".format(len(new_links)))

# Remove links from original graph


Before removing duplications: 225020
Number of links recontructed from ULTRA: 70737


225020